# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [50]:
customer=data.groupby(['CustomerID','ProductName'],axis=0).agg({'Quantity':'sum'})

In [51]:
customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [56]:
table=customer.pivot_table(index='ProductName',columns='CustomerID',fill_value=0)
table.head()

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [59]:
distances = pdist(table.T, 'euclidean')
print(squareform(distances))

[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


In [93]:
#maximizing the resolution of the numbers
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=list(map(lambda x: x[1],table.columns)), columns=list(map(lambda x: x[1],table.columns)))

distances.head()

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [255]:
similarities=distances[33].sort_values(ascending=False)
similarities[1:6].index

Int64Index([264, 3535, 3317, 2503, 3305], dtype='int64')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [257]:
similar_customer=customer.loc[list(similarities[1:6].index)]
similar_customer

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

In [198]:
recomend={}
for k,v in dict(similarities[1:6]).items():
    recomend[k]=list(map(lambda x: x*v,list(customer.loc[k]['Quantity'])))
#recomendations=pd.DataFrame(recomend,index=list(customer.loc[k]['Quantity'].index))
#recomendations
#recomendations['Total']=recomendations.sum(axis=1)
#recomendations.sort_values('Total',ascending=False).head()
print(len(recomend[264]))
print(len(recomend[3535]))

62
55


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [262]:
similar_customer2=similar_customer['Quantity'].reset_index()
similar_customer2.head()
product=similar_customer2[['ProductName','Quantity']].groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
product.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [300]:
chosen=customer.loc[[33]].reset_index()
chosen
merged=pd.merge(chosen,product,how='right',on='ProductName')
notpurchased=merged.loc[pd.isnull(merged['Quantity_x'])]
top5_notpurch=notpurchased.sort_values('Quantity_y',ascending=False)[:6]
top5_notpurch

,CustomerID,ProductName,Quantity_x,Quantity_y
31,NaN,Butter - Unsalted,NaN,3
33,NaN,Soup - Campbells Bean Medley,NaN,3
34,NaN,Wine - Blue Nun Qualitatswein,NaN,3
32,NaN,Wine - Ej Gallo Sierra Valley,NaN,3
65,NaN,Beans - Wax,NaN,2
57,NaN,Scallops 60/80 Iqf,NaN,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [303]:
costum={}
cost_id=list(set(data['CustomerID']))
for c in cost_id:
    similarities=distances[c].sort_values(ascending=False)
    similar_customer=customer.loc[list(similarities[1:6].index)]
    similar_customer2=similar_customer['Quantity'].reset_index()
    product=similar_customer2[['ProductName','Quantity']].groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
    chosen=customer.loc[[c]].reset_index()
    merged=pd.merge(chosen,product,how='right',on='ProductName')
    notpurchased=merged.loc[pd.isnull(merged['Quantity_x'])]
    top5_notpurch=notpurchased.sort_values('Quantity_y',ascending=False)[:6]
    costum[c]=list(top5_notpurch['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [307]:
all_cust=pd.DataFrame(costum).T
all_cust.head()

,0,1,2,3,4,5
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners,Puree - Passion Fruit
59399,"Shrimp - Baby, Warm Water",Towels - Paper / Kraft,Knife Plastic - White,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach,Cookie Chocolate Chip With
92168,Cup - Translucent 7 Oz Clear,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Spinach - Baby,Pop Shoppe Cream Soda,Bread - Calabrese Baguette
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold,"Chicken - Leg, Boneless"
18441,Soupfoamcont12oz 112con,Beef - Texas Style Burger,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [325]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cosine'))), 
                         index=list(map(lambda x: x[1],table.columns)), columns=list(map(lambda x: x[1],table.columns)))

distances2.head()

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [326]:
costum={}
cost_id=list(set(data['CustomerID']))
for c in cost_id:
    similarities=distances2[c].sort_values(ascending=False)
    similar_customer=customer.loc[list(similarities[1:6].index)]
    similar_customer2=similar_customer['Quantity'].reset_index()
    product=similar_customer2[['ProductName','Quantity']].groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
    chosen=customer.loc[[c]].reset_index()
    merged=pd.merge(chosen,product,how='right',on='ProductName')
    notpurchased=merged.loc[pd.isnull(merged['Quantity_x'])]
    top5_notpurch=notpurchased.sort_values('Quantity_y',ascending=False)[:6]
    costum[c]=list(top5_notpurch['ProductName'])

In [327]:
all_cust2=pd.DataFrame(costum).T
all_cust2.head()

,0,1,2,3,4,5
83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn,"Cheese - Brie,danish"
59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Broom - Corn,Water - Aquafina Vitamin,Cake - Mini Cheesecake
92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round,Nantucket - Pomegranate Pear
49159,"Turnip - White, Organic",Beer - Blue,Peas - Frozen,Lamb - Ground,Beef - Ground Medium,"Beef - Chuck, Boneless"
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%,Tea - Jasmin Green
